In [ ]:
# Import necessary libraries
import pandas as pd
import gdown

# Google Drive file id
file_id = '1aCAQpYi8yunjHlQtqPjodSwfxDGiJ9cL'
url = f'https://drive.google.com/uc?id={file_id}'

# Download the file
gdown.download(url, 'maven_toys_transactions.csv', quiet=False)

# Load the data into a pandas DataFrame
data = pd.read_csv('maven_toys_transactions.csv')

# Display the first few rows of the DataFrame
data.head()

In [ ]:
!pip install gdown

In [ ]:
# Import necessary libraries
import pandas as pd
import gdown

# Google Drive file id
file_id = '1aCAQpYi8yunjHlQtqPjodSwfxDGiJ9cL'
url = f'https://drive.google.com/uc?id={file_id}'

# Download the file
gdown.download(url, 'maven_toys_transactions.csv', quiet=False)

# Load the data into a pandas DataFrame
data = pd.read_csv('maven_toys_transactions.csv')

# Display the first few rows of the DataFrame
data.head()

In [ ]:
# Check the shape of the DataFrame
data.shape

In [ ]:
# Check the data types and missing values
data.info()

In [ ]:
# Fill missing values in 'Customer_Satisfaction' with the median
data['Customer_Satisfaction'].fillna(data['Customer_Satisfaction'].median(), inplace=True)

# Check if there are any missing values left
data.isnull().sum()

In [ ]:
# Convert 'Date' and 'Store_Open_Date' to datetime type
data['Date'] = pd.to_datetime(data['Date'])
data['Store_Open_Date'] = pd.to_datetime(data['Store_Open_Date'])

# Check the data types again to confirm
data.dtypes

In [ ]:
# Find the top 10 best-selling products
top_products = data.groupby('Product_Name')['Units'].sum().sort_values(ascending=False).head(10)
top_products

In [ ]:
# Find the most popular product categories
top_categories = data.groupby('Product_Category')['Units'].sum().sort_values(ascending=False)
top_categories

In [ ]:
# Compare the sales of the stores in Guadalajara to the sales of stores in other areas
area_sales = data.groupby('Store_Location')['Units'].sum().sort_values(ascending=False)
area_sales

In [ ]:
# Check the unique values in the 'Store_Name' column
data['Store_Name'].unique()

In [ ]:
# Extract the city name from the 'Store_Name' column
data['City'] = data['Store_Name'].str.split(' ').str[2]

# Compare the sales of the stores in Guadalajara to the sales of stores in other cities
city_sales = data.groupby('City')['Units'].sum().sort_values(ascending=False)
city_sales

In [ ]:
# Compare the sales of the individual stores in Guadalajara
guadalajara_stores = data[data['City'] == 'Guadalajara']
guadalajara_sales = guadalajara_stores.groupby('Store_Name')['Units'].sum().sort_values(ascending=False)
guadalajara_sales

In [ ]:
# Calculate the average customer satisfaction
avg_satisfaction = data['Customer_Satisfaction'].mean()
avg_satisfaction

In [ ]:
# Import necessary libraries
import matplotlib.pyplot as plt

# Remove the dollar sign from the 'Product_Price' column and convert it to float
data['Product_Price'] = data['Product_Price'].str.replace('$', '').astype(float)

# Create a scatter plot of 'Product_Price' and 'Customer_Satisfaction'
plt.figure(figsize=(10, 6))
plt.scatter(data['Product_Price'], data['Customer_Satisfaction'], alpha=0.5)
plt.title('Customer Satisfaction vs Product Price')
plt.xlabel('Product Price')
plt.ylabel('Customer Satisfaction')
plt.show()

In [ ]:
# Calculate the average customer satisfaction for each payment type
payment_satisfaction = data.groupby('Sale_Payment_Type')['Customer_Satisfaction'].mean().sort_values(ascending=False)
payment_satisfaction

In [ ]:
# Extract the month and year from the 'Date' column
data['YearMonth'] = data['Date'].dt.to_period('M')

# Calculate the total sales for each month
monthly_sales = data.groupby('YearMonth')['Units'].sum()

# Create a line plot of the total monthly sales
plt.figure(figsize=(10, 6))
monthly_sales.plot(kind='line')
plt.title('Total Monthly Sales')
plt.xlabel('Month')
plt.ylabel('Total Sales')
plt.show()

In [ ]:
# Calculate the monthly sales for each product category
category_sales = data.groupby(['YearMonth', 'Product_Category'])['Units'].sum().unstack()

# Create a line plot of the monthly sales for each product category
plt.figure(figsize=(10, 6))
category_sales.plot(kind='line', ax=plt.gca())
plt.title('Monthly Sales by Product Category')
plt.xlabel('Month')
plt.ylabel('Total Sales')
plt.show()

In [ ]:
# Calculate the monthly sales for each city
city_sales = data.groupby(['YearMonth', 'City'])['Units'].sum().unstack()

# Create a line plot of the monthly sales for each city
plt.figure(figsize=(10, 6))
city_sales.plot(kind='line', ax=plt.gca())
plt.title('Monthly Sales by City')
plt.xlabel('Month')
plt.ylabel('Total Sales')
plt.show()

In [ ]:
# Create a scatter plot of 'Customer_Satisfaction' and 'Units'
plt.figure(figsize=(10, 6))
plt.scatter(data['Units'], data['Customer_Satisfaction'], alpha=0.5)
plt.title('Customer Satisfaction vs Units Sold')
plt.xlabel('Units Sold')
plt.ylabel('Customer Satisfaction')
plt.show()